In [3]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 12.9 MB/s eta 0:00:00


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import nltk
import string
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [5]:
from google.colab import files

uploaded = files.upload()

Saving Datav.csv to Datav.csv


In [6]:
df=pd.read_csv('Datav.csv')
df.head(10)

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,15-01-2023 12:30,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,15-01-2023 08:45,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,15-01-2023 15:45,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,15-01-2023 18:20,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,15-01-2023 19:55,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19
5,5,5,Feeling grateful for the little things in lif...,Positive,16-01-2023 09:10,GratitudeNow,Twitter,#Gratitude #PositiveVibes,25,50,India,2023,1,16,9
6,6,6,Rainy days call for cozy blankets and hot coc...,Positive,16-01-2023 14:45,RainyDays,Facebook,#RainyDays #Cozy,10,20,Canada,2023,1,16,14
7,7,7,The new movie release is a must-watch! ...,Positive,16-01-2023 19:30,MovieBuff,Instagram,#MovieNight #MustWatch,15,30,USA,2023,1,16,19
8,8,8,Political discussions heating up on the timel...,Negative,17-01-2023 08:00,DebateTalk,Twitter,#Politics #Debate,30,60,USA,2023,1,17,8
9,9,9,Missing summer vibes and beach days. ...,Neutral,17-01-2023 12:20,BeachLover,Facebook,#Summer #BeachDays,18,35,Australia,2023,1,17,12


In [7]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Text', 'Sentiment', 'Timestamp', 'User',
       'Platform', 'Hashtags', 'Retweets', 'Likes', 'Country', 'Year', 'Month',
       'Day', 'Hour'],
      dtype='object')

In [8]:
df.drop(['Unnamed: 0.1','Unnamed: 0','Timestamp','User','Platform','Hashtags','Retweets','Likes','Country','Year','Month','Day','Hour'],axis=1,inplace=True)
df.head()

,Text,Sentiment
0,Enjoying a beautiful day at the park! ...,Positive
1,Traffic was terrible this morning. ...,Negative
2,Just finished an amazing workout! 💪 ...,Positive
3,Excited about the upcoming weekend getaway! ...,Positive
4,Trying out a new recipe for dinner tonight. ...,Neutral


In [9]:
df.shape

(732, 2)

In [10]:
df['Sentiment'].value_counts()

,count
Sentiment,
Positive,44
Joy,42
Excitement,32
Neutral,14
Contentment,14
...,...
Adrenaline,1
Harmony,1
ArtisticBurst,1


In [11]:
df['Sentiment'].value_counts()

,count
Sentiment,
Positive,44
Joy,42
Excitement,32
Neutral,14
Contentment,14
...,...
Adrenaline,1
Harmony,1
ArtisticBurst,1


In [12]:
dict(df['Sentiment'].value_counts())

{' Positive  ': 44,
 ' Joy ': 42,
 ' Excitement ': 32,
 ' Neutral ': 14,
 ' Contentment ': 14,
 ' Happy ': 14,
 ' Hopeful ': 9,
 ' Sad ': 9,
 ' Gratitude ': 9,
 ' Curiosity ': 8,
 ' Embarrassed ': 8,
 ' Loneliness ': 7,
 ' Elation       ': 6,
 ' Bad ': 6,
 ' Despair ': 6,
 ' Hate ': 6,
 ' Playful ': 6,
 ' Gratitude     ': 5,
 ' Curiosity       ': 5,
 ' Enthusiasm    ': 5,
 ' Inspired ': 5,
 ' Empowerment   ': 5,
 ' Contentment   ': 5,
 ' Serenity        ': 5,
 ' Determination   ': 5,
 ' Acceptance      ': 5,
 ' Ambivalence     ': 5,
 ' Nostalgia       ': 5,
 ' Melancholy      ': 5,
 ' Numbness        ': 5,
 ' Confusion       ': 5,
 ' Serenity      ': 5,
 ' Indifference    ': 5,
 ' Frustrated ': 5,
 ' Bitterness ': 5,
 ' Euphoria      ': 4,
 ' Grief ': 4,
 ' Betrayal ': 4,
 ' Frustration ': 4,
 ' Nostalgia ': 4,
 ' Awe ': 4,
 ' Inspiration ': 4,
 ' Negative  ': 4,
 ' Proud ': 4,
 ' Grateful ': 4,
 ' Compassionate ': 4,
 ' Desolation ': 4,
 ' Hope          ': 4,
 ' Neutral   ': 4,
 ' Exc

In [13]:
df['Sentiment'].isnull()

,Sentiment
0,False
1,False
2,False
3,False
4,False
...,...
727,False
728,False
729,False
730,False


In [14]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [15]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


In [16]:
## We shall perform text preprocessing on our data for better results

def transform_emojis(text):
    return emoji.demojize(text)

def preprocess_text(text):
    if not isinstance(text, str):
        return text

    # Transform emojis
    text = transform_emojis(text)

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)

    # Tokenize
    tokens = word_tokenize(text)

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Remove numbers
    tokens = [token for token in tokens if not token.isdigit()]

    return tokens

df['PrText']=df['Text'].apply(preprocess_text)
#After transformation here's how it looks
df.head()

,Text,Sentiment,PrText
0,Enjoying a beautiful day at the park! ...,Positive,"[enjoying, beautiful, day, park]"
1,Traffic was terrible this morning. ...,Negative,"[traffic, terrible, morning]"
2,Just finished an amazing workout! 💪 ...,Positive,"[finished, amazing, workout, flexedbiceps]"
3,Excited about the upcoming weekend getaway! ...,Positive,"[excited, upcoming, weekend, getaway]"
4,Trying out a new recipe for dinner tonight. ...,Neutral,"[trying, new, recipe, dinner, tonight]"


In [17]:
#Since we cannot feed this into the Model,we have to convert it into vectors
!pip install gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

def vectorize(tokens,model):
  valid_tokens = [token for token in tokens if token in model.wv.index_to_key]
  if not valid_tokens:
        return np.zeros(model.vector_size)
  return np.mean([model.wv[token] for token in valid_tokens], axis=0)
model=Word2Vec(df['PrText'].tolist(),vector_size=100, window=5, min_count=1)
df['Embeddings']=df['PrText'].apply(lambda x: vectorize(x, model))
df['Embeddings'].head()

,Embeddings
0,"[-0.0026879148, 0.0012386004, -0.0020856434, 0..."
1,"[0.002310421, -0.0026913497, 0.0024138791, 0.0..."
2,"[-0.0030862773, -0.00022521582, -2.8910403e-05..."
3,"[0.003672516, 0.0070619164, -0.002378391, -0.0..."
4,"[-0.001204375, 0.001831765, -0.00014213212, 0...."


In [18]:
model=Word2Vec(df['PrText'].tolist(),vector_size=100, window=5, min_count=1)
df['Embeddings']=df['PrText'].apply(lambda x: vectorize(x, model))
df['Embeddings'].head()

,Embeddings
0,"[-0.0026879148, 0.0012386004, -0.0020856434, 0..."
1,"[0.002310421, -0.0026913497, 0.0024138791, 0.0..."
2,"[-0.0030862773, -0.00022521582, -2.8910403e-05..."
3,"[0.003672516, 0.0070619164, -0.002378391, -0.0..."
4,"[-0.001204375, 0.001831765, -0.00014213212, 0...."


In [19]:
#Let's look at the number of unique sentiments
df['Sentiment'].unique()

array([' Positive  ', ' Negative  ', ' Neutral   ', ' Anger        ',
       ' Fear         ', ' Sadness      ', ' Disgust      ',
       ' Happiness    ', ' Joy          ', ' Love         ',
       ' Amusement    ', ' Enjoyment    ', ' Admiration   ',
       ' Affection    ', ' Awe          ', ' Disappointed ',
       ' Surprise     ', ' Acceptance   ', ' Adoration    ',
       ' Anticipation ', ' Bitter       ', ' Calmness     ',
       ' Confusion    ', ' Excitement   ', ' Kind         ',
       ' Pride        ', ' Shame        ', ' Confusion ', ' Excitement ',
       ' Shame ', ' Elation       ', ' Euphoria      ', ' Contentment   ',
       ' Serenity      ', ' Gratitude     ', ' Hope          ',
       ' Empowerment   ', ' Compassion    ', ' Tenderness    ',
       ' Arousal       ', ' Enthusiasm    ', ' Fulfillment  ',
       ' Reverence     ', ' Compassion', ' Fulfillment   ', ' Reverence ',
       ' Elation   ', ' Despair         ', ' Grief           ',
       ' Loneliness     

In [20]:
!pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


analyzer = SentimentIntensityAnalyzer()

def get_vader_sentiment(sentiment):
    score = analyzer.polarity_scores(sentiment)
    if score['compound'] >= 0.05:
        return 'Positive'
    elif score['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'
sentiments=df['Sentiment'].tolist()
sentiments=[get_vader_sentiment(sentiment) for sentiment in sentiments]
df['Category']=pd.Series(sentiments)
df['Category'].value_counts()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 6.3 MB/s eta 0:00:00


,count
Category,
Positive,378
Negative,183
Neutral,171


In [21]:
#Replace positives with 1 neutral with 0 and negaitves with -1
def label(category):
  if category == 'Positive':
    return 1
  elif category == 'Neutral':
    return 0
  else:
    return 1
df['Category']=df['Category'].apply(lambda x: label(x))
X=np.vstack(df['Embeddings'])
y=df['Category']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
classifier = LogisticRegression(solver='newton-cg').fit(X_train, y_train)
y_pred=classifier.predict(X_test)
print(classification_report(y_test,y_pred))
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        40
           1       0.73      1.00      0.84       107

    accuracy                           0.73       147
   macro avg       0.36      0.50      0.42       147
weighted avg       0.53      0.73      0.61       147

Accuracy: 0.7278911564625851


/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:44: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py: